In [49]:
import geopandas as gpd
import requests

def get_isochrone(lat: float, lon: float, duration: int, arrival_time: str) -> gpd.GeoDataFrame:
    """Get isochrone from the isochrone API.
    Args: 
        lat: Latitude of the origin point
        lon: Longitude of the origin point
        duration: Duration of the isochrone in seconds
    """


    request = f"http://localhost:8000/isochrone?lat={lat}&lon={lon}&arrival_time={arrival_time}&duration={duration}"
    response = requests.get(request)

    response.raise_for_status()

    geojson = response.text
    gdf = gpd.read_file(geojson)
    return gdf
        

# Manchester Piccadilly
lat = 53.477
lon = -2.23
duration = 1800
time = "08:30:00"

gdf = get_isochrone(lat, lon, duration, time)
gdf.explore()

In [50]:
import pandas as pd
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

async def get_isochrone_async(session: aiohttp.ClientSession, lat: float, lon: float, duration: int, arrival_time: str) -> gpd.GeoDataFrame:
    """Get isochrone from the isochrone API.
    Args: 
        lat: Latitude of the origin point
        lon: Longitude of the origin point
        duration: Duration of the isochrone in seconds
    """

    request = f"http://localhost:8000/isochrone?lat={lat}&lon={lon}&arrival_time={arrival_time}&duration={duration}"
    async with session.get(request) as response:
        response.raise_for_status()
        geojson = await response.text()
        gdf = gpd.read_file(geojson)
        return gdf
    
station_coords = {'Manchester Piccadilly': (53.477, -2.23),
                  'Manchester Victoria': (53.487, -2.242),
                  'Manchester Oxford Road': (53.474, -2.241),
                  'Bolton': (53.574, -2.425),
                  'Manchester Airport': (53.365, -2.272),
                  'Wigan North Western': (53.543, -2.623),}

async def get_isochrones_async(station_coords: dict, duration: int, arrival_time: str) -> gpd.GeoDataFrame:
    """Get isochrones for a list of stations asynchronously.
    Args:
        station_coords: Dictionary of station names and coordinates
        duration: Duration of the isochrones in seconds
    """
    async with aiohttp.ClientSession() as session:
        tasks = [get_isochrone_async(session, lat, lon, duration, arrival_time) for lat, lon in station_coords.values()]
        isochrones = await asyncio.gather(*tasks)
        isochrones = pd.concat(isochrones, ignore_index=True)
        isochrones['station'] = station_coords.keys()
        return isochrones


def get_plot():
    isochrones = asyncio.run(get_isochrones_async(station_coords, 1800, "08:30:00"))
    plot = isochrones.explore(column='station')
    return plot

plot = get_plot()
plot